Rapidly Growing Random Trees (RRT)
==================================

**Topic**: Introduction to RRT planners

Version | Author
------------ | -------------
0.2 | Björn Hein


This worksheet **explains the basic concepts of path planning using a RRT planner)** 

License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)


Important links are:

    General Info: http://www.python.org
    Python tutorial http://www.python.org/doc/tut/
    NetworkX http://networkx.github.io/
    NumPy and SciPy Guide http://docs.scipy.org/
    Matplotlib gallery http://matplotlib.sourceforge.net/gallery.html

Remember that:

    you have to press ctrl-return or shift-return, to execute the code in the code sections, only then the variables are "generated" and can be used
    you can execute the whole notebook by Cell->runAll





Algorithm I
--------------

Following an example implementation of a RRT. Parameters are *numberOfGeneratedNodes*, which limits the number of generated nodes to the given amount and *testGoalAfterNumberOfNodes* which indicates after which number of nodes the goal position should be tested for connection.

In [ ]:
import sys
sys.path.append("templates")

%load_ext autoreload
%autoreload 2

In [ ]:
from scipy.spatial import cKDTree
from IPPRMBase import PRMBase
import numpy as np

import networkx as nx
import random


class RRTSimple(PRMBase):

    def __init__(self, _collChecker):
        """
        _collChecker: the collision checker interface
        """
        super(RRTSimple, self).__init__(_collChecker)
        self.graph = nx.Graph()
        self.lastGeneratedNodeNumber = 0

    def planPath(self, startList, goalList, config):
        """
        
        Args:
            start (array): start position in planning space
            goal (array) : goal position in planning space
            config (dict): dictionary with the needed information about the configuration options
            
        Example:
            config["numberOfGeneratedNodes"] = 500 
            config["testGoalAfterNumberOfNodes"]  = 10
        """
        # 0. reset
        self.graph.clear()
        self.lastGeneratedNodeNumber = 0
        
        # 1. check start and goal whether collision free (s. BaseClass)
        checkedStartList, checkedGoalList = self._checkStartGoal(startList,goalList)

        # 2. add start and goal to graph
        self.graph.add_node(self.lastGeneratedNodeNumber, pos=checkedStartList[0])
        self.lastGeneratedNodeNumber +=1
        
        while self.lastGeneratedNodeNumber < config["numberOfGeneratedNodes"]:
                    
            posList = list(nx.get_node_attributes(self.graph,'pos').values())
            kdTree = cKDTree(posList)

            if (self.lastGeneratedNodeNumber % config["testGoalAfterNumberOfNodes"]) == 0:
                #print "testing goal"
                result = kdTree.query(checkedGoalList[0],k=1)
                if not self._collisionChecker.lineInCollision(self.graph.nodes[result[1]]['pos'],checkedGoalList[0]):
                    self.graph.add_node("goal", pos=checkedGoalList[0])
                    self.graph.add_edge(result[1],"goal")
                    mapping={0:'start'}
                    self.graph = nx.relabel_nodes(self.graph,mapping)

                    return nx.shortest_path(self.graph,"start","goal")

            
            pos = self._getRandomFreePosition()

            # for every node in graph find nearest neigbhours

            result = kdTree.query(pos,k=1)
            if None:
                raise Exception("Something went wrong regarding nearest neighbours")
            #print result
            if not self._collisionChecker.lineInCollision(self.graph.nodes[result[1]]['pos'],pos):
                self.graph.add_node(self.lastGeneratedNodeNumber, pos=pos)
                self.graph.add_edge(result[1],self.lastGeneratedNodeNumber)
                self.lastGeneratedNodeNumber +=1
            


Test
=====

In [ ]:

def rrtPRMVisualize(planner, solution, ax = None, nodeSize = 300):
    """ Draw graph, obstacles and solution in a axis environment of matplotib.
    """
    graph = planner.graph
    collChecker = planner._collisionChecker
    pos = nx.get_node_attributes(graph,'pos')
    color = nx.get_node_attributes(graph,'color')
    
    # get a list of positions of all nodes by returning the content of the attribute 'pos'
    pos = nx.get_node_attributes(graph,'pos')
  
    # draw graph (nodes colorized by degree)
    nx.draw(graph, pos, ax = ax, node_size=nodeSize)
     

    collChecker.drawObstacles(ax)
    
    
    # draw nodes based on solution path
    Gsp = nx.subgraph(graph,solution)
    nx.draw_networkx_nodes(Gsp,pos,
                            node_size=nodeSize,
                             node_color='g',  ax = ax)
        
    # draw edges based on solution path
    nx.draw_networkx_edges(Gsp,pos,alpha=0.8,edge_color='g',width=nodeSize/30.0,  ax = ax)
        
    # draw start and goal
    if "start" in graph.nodes(): 
        nx.draw_networkx_nodes(graph,pos,nodelist=["start"],
                                   node_size=nodeSize,
                                   node_color='#00dd00',  ax = ax)
        nx.draw_networkx_labels(graph,pos,labels={"start": "S"},  ax = ax)


    if "goal" in graph.nodes():
        nx.draw_networkx_nodes(graph,pos,nodelist=["goal"],
                                   node_size=nodeSize,
                                   node_color='#DD0000',  ax = ax)
        nx.draw_networkx_labels(graph,pos,labels={"goal": "G"},  ax = ax)

   

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from shapely.geometry import Point, Polygon, LineString
from IPEnvironment import CollisionChecker

# Definition of a testing environment
trapField = dict()
trapField["obs1"] =   LineString([(6, 18), (6, 8), (16, 8), (16,18)]).buffer(1.0)

environment = CollisionChecker(trapField)



mystart = [[1,20]]
myend = [[20,1]]
# RRTSimple
rrtSimple = RRTSimple(environment)
rrtSimpleConfig = dict()
rrtSimpleConfig["numberOfGeneratedNodes"] = 300 
rrtSimpleConfig["testGoalAfterNumberOfNodes"]  = 10


#environment.resetCounter()
solution = rrtSimple.planPath(mystart, myend, rrtSimpleConfig)

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
rrtPRMVisualize(rrtSimple, solution, ax=ax)



Algorithm II
==========

Following example shows an example, how to modify the RRT. In this example the new connection is only generated in the direction of the newly generated node. Other strategies - as discussed in the lecture - are to incorporate control laws modifiying the direction of the newly generated graph connections.

In [ ]:
class RRT(PRMBase):

    def __init__(self, _collChecker):
        """
        _collChecker: the collision checker interface
        """
        super(RRT, self).__init__(_collChecker)
        self.graph = nx.Graph()
        self.lastGeneratedNodeNumber = 0


    def planPath(self, startList, goalList, config):
        """
        
        Args:
            start (array): start position in planning space
            goal (array) : goal position in planning space
            config (dict): dictionary with the needed information about the configuration options
            
        Example:
            config["numberOfGeneratedNodes"] = 500 
            config["testGoalAfterNumberOfNodes"]  = 10
        """
        # 0. reset
        self.graph.clear()
        self.lastGeneratedNodeNumber = 0
        
        # 1. check start and goal whether collision free (s. BaseClass)
        checkedStartList, checkedGoalList = self._checkStartGoal(startList,goalList)
        
        # 2. add start and goal to graph
        self.graph.add_node(self.lastGeneratedNodeNumber, pos=checkedStartList[0])
        self.lastGeneratedNodeNumber +=1
        
        while self.lastGeneratedNodeNumber < config["numberOfGeneratedNodes"]:
                    
            posList = list(nx.get_node_attributes(self.graph,'pos').values())
            kdTree = cKDTree(posList)

            if (self.lastGeneratedNodeNumber % config["testGoalAfterNumberOfNodes"]) == 0:
                #print "testing goal"
                result = kdTree.query(checkedGoalList[0],k=1)
                if not self._collisionChecker.lineInCollision(self.graph.nodes[result[1]]['pos'],checkedGoalList[0]):
                    self.graph.add_node("goal", pos=checkedGoalList[0])
                    self.graph.add_edge(result[1],"goal")
                    mapping={0:'start'}
                    self.graph = nx.relabel_nodes(self.graph,mapping)

                    return nx.shortest_path(self.graph,"start","goal")

            

            pos = self._getRandomFreePosition()
            
            # for every node in graph find nearest neigbhours
            result = kdTree.query(pos,k=1)
            if None:
                raise Exception("Something went wrong regarding nearest neighbours")
            
            start = np.array(self.graph.nodes[result[1]]['pos'])
            end   = np.array(pos)
            newPos = 0.5 * (end-start) + start
            if not self._collisionChecker.lineInCollision(self.graph.nodes[result[1]]['pos'],newPos):
                self.graph.add_node(self.lastGeneratedNodeNumber, pos=newPos)
                self.graph.add_edge(result[1],self.lastGeneratedNodeNumber)
                self.lastGeneratedNodeNumber +=1
            
 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from shapely.geometry import Point, Polygon, LineString
from IPEnvironment import CollisionChecker

# Definition of a testing environment
trapField = dict()
trapField["obs1"] =   LineString([(6, 18), (6, 8), (16, 8), (16,18)]).buffer(1.0)

environment = CollisionChecker(trapField)


mystart = [[1,20]]
myend = [[20,1]]
# RRTSimple
rrt = RRT(environment)
rrtConfig = dict()
rrtConfig["numberOfGeneratedNodes"] = 100
rrtConfig["testGoalAfterNumberOfNodes"]  = 20

solution = rrt.planPath(mystart, myend, rrtConfig)

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
rrtPRMVisualize(rrt.graph, rrt._collisionChecker, solution, ax=ax)

Tasks / Excercises
==================

* Play with parameters (numberOfGeneratedNodes, testGoalAfterNumberOfNodes), change start goal,....
* Test different environments (simpleField, trapField, fatBottleNeckField, bottleNeckField, clutteredField)